# First Attempt at DeepNN for tumor subtype classification

This will need a lot of work so bare with me :3

In [4]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rc

from tqdm import tqdm

from custom_functions import store_json, load_json

In [5]:
# Constants
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

RANDOM_SEED = 42
EPOCHS = 200
BATCH_SIZE = 16

HAPPY_COLORS_PALETTE =\
["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

## Data

In [19]:
dfr = pd.read_csv("./data/Train_call.txt", sep='\t')
dfr_sols = pd.read_csv("./data/Train_clinical.txt", sep='\t')

df_instances = dfr.T

df_instances["Subgroup"] = None if type(dfr_sols.Subgroup.values[-1]) != int else 0

for arr_name in dfr_sols["Sample"]:
    df_instances.at[arr_name, "Subgroup"] =\
     dfr_sols[dfr_sols["Sample"] == arr_name]["Subgroup"].values[0]

if None in df_instances.Subgroup:
    df_instances.Subgroup.replace({None: "dummy"}, inplace=True)

In [23]:
df_instances.to_csv("./data/fastai_superset.csv")

In [24]:
df_instances

,0,1,2,3,4,5,6,7,8,9,...,2825,2826,2827,2828,2829,2830,2831,2832,2833,Subgroup
Chromosome,1,1,1,1,1,1,1,1,1,1,...,23,23,23,23,23,23,23,23,23,dummy
Start,2927,85022,370546,471671,792533,912799,1271190,1676445,1738295,2481927,...,152422390,152552851,152576854,152994680,153062077,153466463,153504394,153938998,153997146,dummy
End,43870,216735,372295,786483,907406,1266212,1590570,1703748,2477597,2562342,...,152548587,152570071,152935130,153054487,153452633,153491568,153933426,153989329,154492924,dummy
Nclone,3,4,4,5,13,96,45,4,104,14,...,20,6,64,5,57,4,55,5,43,dummy
Array.129,0,0,0,0,0,0,0,0,0,0,...,2,2,2,0,1,1,1,1,1,HER2+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,HER2+
Array.123,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,HR+
Array.100,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,HR+
Array.134,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,HR+


In [8]:
# Create a tensor viable df
# e.g.:
# df['thal'] = pd.Categorical(df['thal'])
# df['thal'] = df.thal.cat.codes

dfc = df.drop(["Unnamed: 0"], axis=1).copy()

for item in dfc:
    dfc[item] = pd.Categorical(dfc[item])
    dfc[item] = dfc[item].cat.codes

In [10]:
from sklearn.model_selection import train_test_split

X = dfc.drop(["Subgroup"], axis=1)
y = dfc.Subgroup

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)

In [11]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())

X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

## DeepNN

In [2]:
%%capture
!pip install fastai

In [3]:
from fastai.tabular.all import *

/root/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
path = Path("./data/")

In [25]:
dls = TabularDataLoaders.from_csv(path/'fastai_superset.csv', path=path, y_names="Subgroup",
    # cat_names = [],
    # cont_names = [],
    procs = [Categorify, FillMissing, Normalize])

ValueError: Unable to coerce to Series, length must be 2835: given 2834

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [ ]:
# path = untar_data(URLs.ADULT_SAMPLE)

# dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
#     cat_names = ['workclass', 'education', 'marital-status', 'occupation',
#                  'relationship', 'race'],
#     cont_names = ['age', 'fnlwgt', 'education-num'],
#     procs = [Categorify, FillMissing, Normalize])

# learn = tabular_learner(dls, metrics=accuracy)
# learn.fit_one_cycle(2)

In [ ]:
from numpy import vstack
from numpy import argmax

from pandas import read_csv

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from torch import Tensor, from_numpy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    def reselect(self, selection):
        # Return only the selected features
        self.X = self.X.include(selection)
        # Should work fine without this step
        self.X = self.X.astype('float32')
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        X = self.act3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(500):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

            
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [ ]:
from torch import nn

class NN(nn.Module):
  """"""

  def __init__(self, n_features):
    """Return Self"""
    super(NN, self).__init__()

    self.fc1 = nn.Linear(n_features, 1200)
    self.fc2 = nn.Linear(1200, 600)
    self.fc3 = nn.Linear(600, 300)
    self.fc4 = nn.Linear(300, 3)

  def forward(self, x):
    """Forward Pass for training"""
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))

    return torch.sigmoid(self.fc4(x))

DeepNN = NN(X_train.shape[1])

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)

def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)


for epoch in range(EPOCHS):
    y_pred = DeepNN(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % 100 == 0:
      train_acc = calculate_accuracy(y_train, y_pred)
      y_test_pred = net(X_test)
      y_test_pred = torch.squeeze(y_test_pred)
      test_loss = criterion(y_test_pred, y_test)
      test_acc = calculate_accuracy(y_test, y_test_pred)
      print(f'''
      epoch {epoch}
      Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
      Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
      ''')
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

ValueError: Using a target size (torch.Size([70])) that is different to the input size (torch.Size([70, 3])) is deprecated. Please ensure they have the same size.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c915e4f9-60c2-40b5-a522-8a90cb3fd50a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>